# Infer

First, make paragraphs from lines (unwrap), then wrap to the limits of the network

In [1]:
import sys, os, datetime
import json
import torch
import random
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TranslationPipeline
from collections import defaultdict
import zipfile

2023-06-05 12:15:44.508388: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 12:15:45.165977: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-05 12:15:46.257469: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.7/lib64:
2023-06-05 12:15:46.257577: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [2]:
import languages

In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
src_langs = set(["akk", "sux"])
tgt_langs = set(["en"])

In [5]:
model_id = "praeclarum/cuneiform"
# model_revision = "1ba74c8dcf6d1839b0a56589a53dfb5c20ca84f2"
model_revision = "7a60be19efe61bf4adf873eb86f864ea7bfb4876"

In [6]:
batch_size = 16
device = "cuda" if torch.has_cuda else "cpu"
device_id = 0 if device == "cuda" else -1

In [7]:
!nvidia-smi

Mon Jun  5 12:15:52 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.105.01   Driver Version: 515.105.01   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0  On |                  N/A |
|  0%   33C    P8    19W / 350W |    451MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Load Existing Translations

In [8]:
output_json_path = "../data/ml_translations.json"
output_zip_path = output_json_path.replace(".json", ".zip")

In [9]:
with open(output_zip_path, "rb") as zipf:
    with zipfile.ZipFile(zipf) as zf:
        name = [x for x in zf.namelist() if x.endswith(".json")][0]
        with zf.open(name) as f:
            old_translations = json.loads(str(f.read(), "utf8"))
old_translations.keys()
len(old_translations["sux_to_en"])

553291

In [10]:
new_translations = dict(old_translations)
new_translations.keys()


dict_keys(['model_id', 'model_revision', 'akk_to_en', 'sux_to_en'])

In [11]:
if new_translations["model_revision"] != model_revision:
    print("Clearing defunct translations")
    for s_lang in src_langs:
        for t_lang in tgt_langs:
            st_key = f"{s_lang}_to_{t_lang}"
            new_translations[st_key] = {}

In [12]:
def sample_translations():
    for s_lang in src_langs:
        for t_lang in tgt_langs:
            st_key = f"{s_lang}_to_{t_lang}"
            if st_key in new_translations:
                translations = new_translations[st_key]
                print(len(translations), f"{st_key} translations")
                print([(x, translations[x]) for x in translations][:10])

In [13]:
sample_translations()

553291 sux_to_en translations
[('" [...] ~ [...] = %a [u4]-du-ru-u2 " [...] ~ [...] = %a {d}en-lil2 " [...] ~ [...] = %a szu-ub-tum', '... ... ... ...'), ('" [...] ~ |_x-x-lal_| = %a na-du-u2-um " [...] ~ |_x-x-si_| = %a ku-ur-ku-u2-um " [...] ~ |_su-lu-sze3_|# = %a lu-up-pu-um', '... ... ... ... ...'), ('" [ba]-e# = %a za-zum " [za-e] = %a at#-ta " [ga2-e] = %a [a]-na#?-ku-u2 " [...] = %a [at]-ta', 'you are'), ('# " ~ _munsub #_ " ~ _alan# #_ " ~ _alan #_ " ~ |_masz-gu2-gar3_| # zubx(|_masz-gu2-gar3_|) " zu-ub ~ |_masz-gu2-gar3_| = %a zu-bu-um', 'a kind of acquatic animal a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of in

## Load the Model

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_id, revision=model_revision, device=device)
model_max_length = tokenizer.model_max_length
model_max_length

512

In [15]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, revision=model_revision, max_length=tokenizer.model_max_length)
model = model.to(device)
# model

In [16]:
pipeline = TranslationPipeline(model=model, tokenizer=tokenizer, batch_size=batch_size, device=device_id)

In [17]:
print(pipeline("translate Akkadian to English: 1(disz){d}szul3-ma-nu-_sag man gal?_-u2 _man_ dan-nu _man kisz_"))
print(pipeline("translate Akkadian to English: ra-bi-isz e-pu-usz"))

[{'translation_text': 'ulmanu-re, great king, strong king, king of the universe,'}]
[{'translation_text': 'I built it anew.'}]


## Load Transliterations to Translate

In [18]:
import corpi
import cdli

In [19]:
import importlib
importlib.reload(corpi)

<module 'corpi' from '/home/fak/Projects/CuneiformTranslators/tools/corpi.py'>

## Load ORACC

In [20]:
importlib.reload(cdli)

<module 'cdli' from '/home/fak/Projects/CuneiformTranslators/tools/cdli.py'>

In [21]:
oracc_dir = os.path.abspath(f"/Volumes/FrankDisk/oracc_zips")
# oracc_dir = os.path.abspath(f"/home/fak/nn/Data/oracc_zips")
oracc_dir

'/Volumes/FrankDisk/oracc_zips'

In [22]:
oracc_corpus = corpi.ORACC(oracc_dir)

## Load CDLI

In [23]:
cdli_corpus = corpi.CDLI()

## Find just the transliterations

In [24]:
def get_all_transliterated_pubs(src_lang):
    transliterated_cdli_index = cdli_corpus.get_pubs_with_lang(src_lang)
    transliterated_cdli_ids = set(transliterated_cdli_index.keys())
    transliterated_oracc_index = oracc_corpus.get_pubs_with_lang(src_lang)
    transliterated_oracc_ids = set(transliterated_oracc_index.keys())
    all_ids = set(transliterated_oracc_ids)
    all_ids = all_ids.union(transliterated_cdli_ids)
    return all_ids, transliterated_cdli_index, transliterated_oracc_index

all_transliterated_pubs = {
    "sux": get_all_transliterated_pubs("sux"),
    "akk": get_all_transliterated_pubs("akk")
}
print("sux", len(all_transliterated_pubs["sux"][0]))
print("akk", len(all_transliterated_pubs["akk"][0]))

sux 103075
akk 31747


## Build Need Translation List

In [25]:
def get_need_translations(src_lang, encoding="ascii", tgt_lang="en"):
    srcs = set()

    pub_ids, transliterated_cdli_index, transliterated_oracc_index = all_transliterated_pubs[src_lang]
    max_line_length = 0

    for pub_id in tqdm(pub_ids):
        corpus = "cdli" if pub_id in transliterated_cdli_index else "oracc"
        if corpus == "cdli":
            pub = transliterated_cdli_index[pub_id]
        else:
            # print(pub_id, "in oracc")
            pub = transliterated_oracc_index[pub_id]
        for a in pub.text_areas:
            if (corpus == "cdli") and len(a.lines) > 0 and len(a.paragraphs) == 0:
                a.lines_to_paragraphs(src_lang, tgt_lang)
            paras = a.paragraphs_to_lines()
            for p in paras:
                for si,ei,line in p:
                    if len(line) > max_line_length:
                        max_line_length = len(line)
                    srcs.add(line)
    print(src_lang, "max line length", max_line_length)
    return srcs
needs_translation = {     
    "sux": get_need_translations("sux"),
    "akk": get_need_translations("akk"),
}
print(len(needs_translation["sux"]), "sux needs translation")
print(len(needs_translation["akk"]), "akk needs translation")


  0%|          | 0/103075 [00:00<?, ?it/s]

sux max line length 458


  0%|          | 0/31747 [00:00<?, ?it/s]

akk max line length 522
524319 sux needs translation
275751 akk needs translation


In [26]:
list(needs_translation["akk"])[:10]

['[_en_]-ia{*} É{*}—[_dingir_{?}-_mesz_{?}] sza _uru_-kàl-[zi x]-_mesz_{*}# szak-nu#',
 '[_e2 nig2-gur11_ sza e2-sag-il2 ip-tu]-u2# _ku3-sig17 ku3-babbar_ ni-siq#-ti# _na4#-mesz#_ a-na _man_ elam#-ma#{ki#}',
 'be-el-[ti{?}-ia{?} x x x x x x] qi-x#+[x x x x x x x x x]',
 '[_masz-masz_-_igi_ n n ... _ge6 25_ ina _zalag2_ sin ar2 _mul2_]-ar2-sza2-_alla_-sza2-_ulu3 2 kusz3_ sin 1/2 _kusz3_ ana _ulu3 sig ge6 26_ ina _zalag2_ sin ina _igi lugal 2 kusz3#_ sin 1 _kusz3_ ana _ulu3 sig#_',
 'sza la ni-bi ul-tu qer-bi-szu2#-un#',
 'u3 sin-mu-ba-li2-it, di-in _e2 {d}utu [di]-ku5-mesz_ {disz}sin#-i-qi2-sza-am {disz}{d}utu-li-wi-ir!',
 '02 GÚ a{*}-na{*}# [x]-ru{?}# A—_sig_ a-na gu{*}-zip{*}-pi LÚ-_gisz_{*}-_ban_{*}-_tag_{*}',
 '[_igi_ {m}x x x]+x# LÚ{v}-MÁ-_du-du uru_-szi-szil-a-a',
 '{d}na-ra-am-{d}suen lugal a-ga-de3{ki}',
 '_{gesz}esi {gesz}tug2_ {gesz}mu-suk-kan-ni _{gesz}eren {gesz}szur-min3_ a-na mu-szab _lugal_-ti-ia u mul-ta-u-ti be-lu-ti-ia']

In [29]:
tgt_lang = "en"

def get_translations():
    for src_lang in src_langs:
        st_key = f"{src_lang}_to_{tgt_lang}"
        news = new_translations[st_key]
        # print(list(olds)[:10])
        for src in tqdm(needs_translation[src_lang]):
            if src not in news:
                yield (src_lang, tgt_lang, src, f"translate {languages.all_languages[src_lang]} to {languages.all_languages[tgt_lang]}: " + src)

num_need_translate = sum([len(x) for x in needs_translation.values()])
print(f"{num_need_translate:,} to translate")
to_translate = list(get_translations())
print(f"{len(to_translate):,} left to translate")
                
def get_translations_gen():
    for src_lang, tft_lang, s, q in tqdm(to_translate):
        yield q

800,070 to translate


  0%|          | 0/524319 [00:00<?, ?it/s]

  0%|          | 0/275751 [00:00<?, ?it/s]

8,754 left to translate


In [30]:
r = pipeline(get_translations_gen())
for i, tr in enumerate(r):
    src_lang, tft_lang, s, q = to_translate[i]
    st_key = f"{src_lang}_to_{tgt_lang}"
    translations = new_translations[st_key]
    t = tr[0]['translation_text']
    translations[s] = t


  0%|          | 0/8754 [00:00<?, ?it/s]

In [31]:
sample_translations()

553296 sux_to_en translations
[('" [...] ~ [...] = %a [u4]-du-ru-u2 " [...] ~ [...] = %a {d}en-lil2 " [...] ~ [...] = %a szu-ub-tum', '... ... ... ...'), ('" [...] ~ |_x-x-lal_| = %a na-du-u2-um " [...] ~ |_x-x-si_| = %a ku-ur-ku-u2-um " [...] ~ |_su-lu-sze3_|# = %a lu-up-pu-um', '... ... ... ... ...'), ('" [ba]-e# = %a za-zum " [za-e] = %a at#-ta " [ga2-e] = %a [a]-na#?-ku-u2 " [...] = %a [at]-ta', 'you are'), ('# " ~ _munsub #_ " ~ _alan# #_ " ~ _alan #_ " ~ |_masz-gu2-gar3_| # zubx(|_masz-gu2-gar3_|) " zu-ub ~ |_masz-gu2-gar3_| = %a zu-bu-um', 'a kind of acquatic animal a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of insect a kind of in

## Save the Translations

In [32]:
def write_translations(f):
    f.write("{\n")
    f.write(f"\"model_id\":\"{model_id}\",\n")
    f.write(f"\"model_revision\":\"{model_revision}\"")
    for st_key in sorted([x for x in new_translations if not x.startswith("model_")]):
        f.write(f",\n\"{st_key}\":{{\n")
        translations = new_translations[st_key]
        head = ""
        for s in sorted(list(translations.keys())):
            f.write(head)
            f.write(json.dumps(s))
            f.write(": ")
            f.write(json.dumps(translations[s]))
            head = ",\n"
        f.write("}")
    f.write("}\n")
    
# write_translations(sys.stdout)

In [33]:
with open(output_json_path, "wt") as f:
    write_translations(f)

In [37]:
import zipfile

def compress(zip_name, file_to_zip):
    if os.path.exists(zip_name):
        os.unlink(zip_name)
    # Create a new zip file and add files to it
    with zipfile.ZipFile(zip_name, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
        zf.write(file_to_zip)
    os.unlink(file_to_zip)

compress('../data/ml_translations.zip', '../data/ml_translations.json')

In [39]:
!ls -al ../data

total 105132
drwxrwxr-x  2 fak fak     4096 Jun  5 12:26 .
drwxrwxr-x 10 fak fak     4096 Jun  1 13:42 ..
-rw-rw-r--  1 fak fak 24652099 Jun  5 11:54 cdli_pubs.zip
-rw-rw-r--  1 fak fak    56450 Jul 25  2022 dataset_index.json
-rw-rw-r--  1 fak fak 32382385 Jun  5 12:26 ml_translations.zip
-rw-rw-r--  1 fak fak 13461354 Jun  5 11:54 oracc_pubs.zip
-rw-rw-r--  1 fak fak 29155935 May 31 12:30 translations_akk_to_en.jsonl
-rw-rw-r--  1 fak fak  7927025 May 31 12:30 translations_sux_to_en.jsonl


In [40]:
new_json = json.load(open(output_json_path, "rt"))
print(len(new_json["akk_to_en"]))
print(len(new_json["sux_to_en"]))

FileNotFoundError: [Errno 2] No such file or directory: '../data/ml_translations.json'